# Month 1: Excel-style + SQL Project

Project: Retail Sales Analysis — create dataset, do Excel-like cleaning & pivot-table, export to .xlsx, load into SQLite and run SQL queries. rename functions, change variable names, rewrite comments in your style, add extra tests.

In [3]:
# =========================
# Cell 1 — Retail Sales (Excel-like) + SQLite
# Saves: retail_sales.xlsx and retail_sales.csv
# =========================

import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path

# --- 1) Create synthetic retail sales dataset (or replace with your CSV) ---
np.random.seed(42)
n = 500
dates = pd.date_range(end=pd.Timestamp.today(), periods=n).to_series().dt.date
store_ids = np.random.choice(['Store_A','Store_B','Store_C'], n)
products = np.random.choice(['Shirt','Pants','Shoes','Hat','Jacket'], n)
units = np.random.randint(1, 8, size=n)
price_map = {'Shirt': 400, 'Pants':700, 'Shoes':1500, 'Hat':250, 'Jacket':2000}
prices = [price_map[p] * (1 + np.random.uniform(-0.2, 0.2)) for p in products]
revenue = np.round(units * prices, 2)

df = pd.DataFrame({
    'date': dates,
    'store': store_ids,
    'product': products,
    'units_sold': units,
    'unit_price': np.round(prices, 2),
    'revenue': revenue
})

# Quick preview
print("Dataset sample:")
print(df.head())

# --- 2) Excel-style operations (cleaning & pivot) ---
# Example cleaning: ensure no nulls
df = df.dropna().reset_index(drop=True)

# Add month/year columns for aggregation
df['month'] = pd.to_datetime(df['date']).dt.to_period('M').astype(str)
df['year'] = pd.to_datetime(df['date']).dt.year

# Pivot table: total revenue by store x month
pivot_store_month = pd.pivot_table(df, values='revenue', index='store', columns='month', aggfunc='sum', fill_value=0)
print("\nPivot (store x month) shape:", pivot_store_month.shape)

# Another pivot: product-wise summary
prod_summary = df.groupby('product').agg(
    total_units=('units_sold','sum'),
    total_revenue=('revenue','sum'),
    avg_price=('unit_price','mean')
).reset_index()
print("\nProduct summary:")
print(prod_summary)

# --- 3) Export to Excel (multiple sheets) ---
out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)
excel_path = out_dir/"retail_sales.xlsx"
with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='raw_data', index=False)
    pivot_store_month.to_excel(writer, sheet_name='pivot_store_month')
    prod_summary.to_excel(writer, sheet_name='product_summary', index=False)

print(f"\nSaved Excel to: {excel_path}")

# --- 4) Save CSV to be used by Power BI / Tableau later ---
csv_path = out_dir/"retail_sales.csv"
df.to_csv(csv_path, index=False)
print(f"Saved CSV to: {csv_path}")

# --- 5) Load dataset into SQLite and run SQL queries (demonstrates SQL capability) ---
db_path = out_dir/"retail_analysis.db"
conn = sqlite3.connect(db_path)
df.to_sql('sales', conn, if_exists='replace', index=False)

# Example SQL queries:
q1 = "SELECT store, SUM(revenue) as total_revenue FROM sales GROUP BY store ORDER BY total_revenue DESC;"
q2 = "SELECT product, SUM(units_sold) as total_units FROM sales GROUP BY product ORDER BY total_units DESC;"

top_stores = pd.read_sql_query(q1, conn)
top_products = pd.read_sql_query(q2, conn)

print("\nTop stores by revenue:")
print(top_stores)

print("\nTop products by units sold:")
print(top_products)

conn.close()

# -----------------------------
# 1) Rename functions/vars: df -> sales_df, pivot_store_month -> monthly_by_store
# 2) Re-implement pivot using groupby+unstack instead of pd.pivot_table
# 3) Add more edge-case tests (zero-unit rows, negative price)
# 4) Rewrite comments in your own words and add README describing the pipeline

Dataset sample:
                                  date    store product  units_sold  \
2024-04-18 12:59:16.004445  2024-04-18  Store_C   Pants           5   
2024-04-19 12:59:16.004445  2024-04-19  Store_A   Pants           5   
2024-04-20 12:59:16.004445  2024-04-20  Store_C   Shoes           3   
2024-04-21 12:59:16.004445  2024-04-21  Store_C   Pants           3   
2024-04-22 12:59:16.004445  2024-04-22  Store_A   Shirt           6   

                            unit_price  revenue  
2024-04-18 12:59:16.004445      747.58  3737.88  
2024-04-19 12:59:16.004445      572.37  2861.83  
2024-04-20 12:59:16.004445     1552.96  4658.87  
2024-04-21 12:59:16.004445      839.82  2519.46  
2024-04-22 12:59:16.004445      408.93  2453.57  

Pivot (store x month) shape: (3, 17)

Product summary:
  product  total_units  total_revenue    avg_price
0     Hat          359       88996.72   247.946882
1  Jacket          458      923890.00  2024.561009
2   Pants          380      267247.36   703.5586

In [2]:
%pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 4.2 MB/s eta 0:00:00
